In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = [224, 224]

train_path = 'cnn_dataset'
valid_path = 'test'

In [3]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 4s 0us/step


In [5]:
for layer in resnet.layers:
    layer.trainable = False

In [6]:
folders = glob('cnn_dataset/*')

In [7]:
x = Flatten()(resnet.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=prediction)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [10]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [13]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [15]:
training_set = train_datagen.flow_from_directory('cnn_dataset',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 24000 images belonging to 3 classes.


In [16]:
test_set = test_datagen.flow_from_directory('test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 6000 images belonging to 3 classes.


In [19]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
750/750 [==============================] - 1437s 2s/step - loss: 0.7080 - accuracy: 0.7410 - val_loss: 0.5354 - val_accuracy: 0.7913
Epoch 2/5
750/750 [==============================] - 1381s 2s/step - loss: 0.5961 - accuracy: 0.7925 - val_loss: 1.4975 - val_accuracy: 0.5975
Epoch 3/5
750/750 [==============================] - 1454s 2s/step - loss: 0.5549 - accuracy: 0.8055 - val_loss: 0.5699 - val_accuracy: 0.7808
Epoch 4/5
750/750 [==============================] - 1374s 2s/step - loss: 0.5827 - accuracy: 0.8133 - val_loss: 0.5515 - val_accuracy: 0.8045
Epoch 5/5
750/750 [==============================] - 1372s 2s/step - loss: 0.5007 - accuracy: 0.8328 - val_loss: 0.4706 - val_accuracy: 0.8282


In [20]:
from tensorflow.keras.models import load_model
model.save('filemodel.h5')

In [21]:
y_pred = model.predict(test_set)

188/188 [==============================] - 269s 1s/step


In [22]:
y_pred

array([[3.4147340e-07, 2.0566991e-07, 9.9999940e-01],
       [1.4213663e-01, 7.6910222e-01, 8.8761173e-02],
       [1.2241732e-02, 8.4393954e-01, 1.4381871e-01],
       ...,
       [8.0547947e-01, 1.9236960e-01, 2.1509705e-03],
       [5.4384091e-05, 7.8174373e-05, 9.9986744e-01],
       [9.4416767e-01, 5.5786300e-02, 4.6066954e-05]], dtype=float32)

In [23]:
y_pred = np.argmax(y_pred, axis=1)

In [24]:
y_pred

array([2, 1, 1, ..., 0, 2, 0], dtype=int64)

In [25]:
model=load_model('filemodel.h5')

In [29]:
img=image.load_img('test/food/8016.jpg',target_size=(224,224))

In [30]:
x=image.img_to_array(img)

In [31]:
x

array([[[69., 68., 63.],
        [69., 68., 63.],
        [69., 68., 63.],
        ...,
        [87., 49., 48.],
        [87., 49., 48.],
        [87., 49., 48.]],

       [[69., 68., 63.],
        [69., 68., 63.],
        [69., 68., 63.],
        ...,
        [87., 49., 48.],
        [87., 49., 48.],
        [87., 49., 48.]],

       [[69., 68., 63.],
        [69., 68., 63.],
        [69., 68., 63.],
        ...,
        [87., 49., 48.],
        [87., 49., 48.],
        [87., 49., 48.]],

       ...,

       [[35., 30., 36.],
        [35., 30., 36.],
        [35., 30., 36.],
        ...,
        [49., 47., 34.],
        [49., 47., 34.],
        [49., 47., 34.]],

       [[35., 30., 36.],
        [35., 30., 36.],
        [35., 30., 36.],
        ...,
        [49., 47., 34.],
        [49., 47., 34.],
        [49., 47., 34.]],

       [[35., 30., 36.],
        [35., 30., 36.],
        [35., 30., 36.],
        ...,
        [49., 47., 34.],
        [49., 47., 34.],
        [49., 47., 34.]]

In [32]:
x.shape

(224, 224, 3)

In [33]:
x=x/255

In [34]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [35]:
model.predict(img_data)

1/1 [==============================] - 1s 1s/step


array([[5.7670243e-02, 8.3665421e-09, 9.4232976e-01]], dtype=float32)

In [36]:
a=np.argmax(model.predict(img_data), axis=1)

1/1 [==============================] - 0s 127ms/step


In [37]:
a==1

array([False])